<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Submit an Existing Notebook to AzureML
In this noteboook we provide a scaffold to directly submit an existing notebook to the AzureML compute targets. Now we don't need to rewrite the training script, instead, we can now take the same notebook we ran on our local machine (01_training_introduction.ipynb), send it to the AzureML platform, and have it run over there. 
        In a couple of minutes, this process generates a copy of our notebook with the results of each cell, that we can download on our machine. This is convenient especially when the computations we want to execute require a lot of memory or even GPUs.

No changes to our existing notebook are required. Now, if we want to be able to track some metrics on the Azure portal, the only thing we need to do is to add "papermill.record() or scrapbook.glue()" to our notebook 

For example, we can add pm.record('training_loss', training_losses) in our training notebook to log training_loss

### Advantages of using AzureML
AzureML lets us:
- Manage cloud resources for monitoring, logging, and organizing our machine learning experiments
- Train models either locally or by using cloud resources, including GPU-accelerated model training
- Scale out easily when dataset grows by just creating and pointing to new compute target

### Prerequisities
For this notebook to run properly on our machine, we will need to have an Azure workspace. If we have not done so yet, we can run through the short [20_azure_workspace_setup.ipynb](https://github.com/microsoft/ComputerVision/blob/master/classification/notebooks/20_azure_workspace_setup.ipynb) notebook to create one.

# Setup environment

### Install azure.contrib.notebook package
We need notebook_run_config from azureml.contrib.notebook to run this notebook. Since azureml.contrib.notebook contains experimental components, it's not included in environment.yml file by default.

In [ ]:
# pip install "azureml.contrib.notebook>=1.0.21.1"

In [1]:
import azureml.core
from azureml.contrib.notebook.notebook_run_config import NotebookRunConfig
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_GPU_IMAGE
from azureml.core.runconfig import RunConfiguration
from azureml.widgets import RunDetails

print("azureml.core version: {}".format(azureml.core.VERSION))

azureml.core version: 1.0.33


### Connect to an AzureML workspace

An [AzureML Workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py) is an Azure resource that organizes and coordinates the actions of many other Azure resources to assist in executing and sharing machine learning workflows. In particular, an Azure ML Workspace coordinates storage, databases, and compute resources providing added functionality for machine learning experimentation, deployment, inferencing, and the monitoring of deployed models.

If we already have a configuration file, setup() extracts the information it needs from it. If we don't, setup() enters an interactive mode and asks for the following pieces of information:
- <b>subscription ID:</b> the ID of the Azure subscription we are using
- <b>resource group:</b> the name of the resource group in which our workspace resides
- <b>workspace region:</b> the geographical area in which our workspace resides (e.g. "eastus2" -- other examples are available [here](https://azure.microsoft.com/en-us/global-infrastructure/geographies/) <i>-- note the lack of spaces</i>)
- <b>workspace name:</b> the name of the workspace we want to create or retrieve.


<i><b>Note:</b> The first time we use this functionality, a webbrowser page may get triggered, on which we will need to choose the email we want to use to connect to our Azure account.</i>

In [2]:
ws = Workspace.setup()

### Create or Attach Azure Machine Learning Compute 

To be able to run our notebook remotely, we need to determine on which machine(s) it will run. For this, we create or retrieve computational resources. This is typically a cluster of CPUs or a cluster of GPUs. Such cluster is also called a [compute target](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets). In the example below, we spin up a GPU cluster with up to 2 nodes. We use [Standard NC6](https://docs.microsoft.com/en-us/azure/virtual-machines/windows/sizes-gpu) machines, which have 1 GPU each. If a cluster with the same name already exists in your workspace, the script loads it instead

In [3]:
# Remote compute (cluster) configuration.
VM_SIZE = 'STANDARD_NC6'
# Cluster nodes
MIN_NODES = 0
MAX_NODES = 2

CLUSTER_NAME = 'gpuclusternc6'

try:
    # Retrieve an already existing cluster associated with this workspace
    compute_target = ComputeTarget(workspace=ws, name=CLUSTER_NAME)
    print("Found existing compute target")
except:
    print("Creating a new compute target...")
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(
        vm_size=VM_SIZE,
        min_nodes=MIN_NODES,
        max_nodes=MAX_NODES
    )
    # Create the cluster with the specified name and configuration
    compute_target = ComputeTarget.create(ws, CLUSTER_NAME, compute_config)
    # Wait for the cluster creation to complete, show the output log
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Found existing compute target


### Configure the Docker image

The notebook here will run in a container, the Docker image is the default setting for Azure GPU. We install the necessary conda dependencies through a `RunConfiguration` to run this notebook sucessfully.  

<b>IMPORTANT</b>: You must have Docker engine installed locally in order to use this execution mode. If your kernel is already running in a Docker container, such as Azure Notebooks, this mode will NOT work.

We also create a `NotebookRunConfig` object, which we will use to run our notebook remotely. This object refers to:
- The `notebook` we are interested in, and to the `source_directory` in which it is currently saved, on our local machine
- The name of the output notebook that will be generated remotely -- This notebook will contain the same cells as the `01_training_introduction.ipynb` notebook, as well as the results that will have been computed on our remote GPU cluster
- The Docker image configuration and other setup parameters we defined in the `run_config` object.

In [6]:
# Configure the Docker image to use
NOTEBOOK_NAME = '01_training_introduction.ipynb' # notebook we want to run on AzureML

run_config = RunConfiguration()
run_config.target = "gpuclusternc6"
run_config.environment.docker.enabled = True
run_config.environment.docker.base_image = DEFAULT_GPU_IMAGE
# This argument can be DEFAULT_GPU_IMAGE or DEFAULT_CPU_IMAGE, 
# depending on the type of machine used to create the cluster (cf. VM_SIZE)

run_config.environment.python.user_managed_dependencies = False
run_config.auto_prepare_environment = True
run_config.environment.python.conda_dependencies = CondaDependencies.create('python_version=3.6.2', pip_packages=['matplotlib', 'scrapbook', 'fastai','sklearn','scikit-learn','bqplot','torch','azureml-sdk'])

# Define the object that will be submitted for execution on the GPU cluster
cfg = NotebookRunConfig(source_directory='../../',
                        notebook='classification/notebooks/' + NOTEBOOK_NAME, # Path of notebook to run
                        output_notebook='outputs/out.ipynb', # Path and name of the output notebook
                        run_config=run_config)

# Run the notebook remotely

To run our notebook, we need to first create an experiment. An experiment contains a series of trials called `Runs`. A run typically contains some tasks, such as training a model, running a script remotely, etc. Through a run's methods, we can log several metrics such as training and test loss and accuracy, as we will see below. The full description of the run class is available [here](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run.run?view=azure-ml-py).

Our experiment takes in our workspace and experiment name as inputs. We then submit the `NotebookRunConfig` object we created above, to our experiment, though the `submit()` method. This constitutes a `run`. This submission triggers the execution of our notebook `01_training_introduction.ipynb` on our remote GPU cluster.

As the notebook gets run remotely, we can keep track of progress through the `RunDetails()` widget.

In [7]:
# Define an experiment
experiment_name = NOTEBOOK_NAME.strip(".ipynb")
exp = Experiment(workspace=ws, name=experiment_name)

# Submit the job
run = exp.submit(cfg)

# Track the run's progress
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

The figure below shows an example rendering of the widget. We can see charts automatically generated from the metrics logged in the last cell of [01_training_introduction.ipynb](01_training_introduction.ipynb); here training accuracy and loss. The widget also give us a link to the portal, so we can view the results there as well.

<img src="media/widget.PNG" width="800" alt="Widget results for papermill notebook run">

We can also download the resulting `out.ipynb` notebook directly from the portal, by clicking on the `Outputs` tab.

<img src="media/output.PNG" width="800" alt="Output.ipynb generated for papermill notebook run">

If we prefer to look at the logged metrics from our notebook, we can do so by using the `get_metrics()` method.

<i><b>Note:</b> The run must be completed to avoid getting an empty object.</i>

In [ ]:
# Retrieve metrics logged during the run
metrics = run.get_metrics()
print(metrics)

# Deprovision compute resource
To avoid unnecessary charges, and if we created a compute target that doesn't scale down to 0 (cf. `MIN_NODES` parameter), we should deprovision it using the `delete()` method.

To get a better sense of pricing, we can refer to [this calculator](https://azure.microsoft.com/en-us/pricing/calculator/?service=virtual-machines). We can also navigate to the [Cost Management + Billing pane](https://ms.portal.azure.com/#blade/Microsoft_Azure_Billing/ModernBillingMenuBlade/Overview) on the portal, click on our subscription ID, and click on the Cost Analysis tab to check our credit usage.

<b>Notes:</b>
- It is important to wait until our run is completed.
- This deletion will take a few minutes. We can check progress in Azure Portal, on the `Compute` tab.

In [ ]:
# compute_target.delete()